<a href="https://colab.research.google.com/github/Davido-Elbosso/Whisper_Ewe/blob/main/Whisper_fine_tune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets evaluate jiwer transformers[torch]

In [ ]:
# from datasets import load_dataset

# ds = load_dataset("dodziraynard/ewe")

In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
from datasets import load_dataset, DatasetDict

# Charger le dataset depuis Hugging Face
# dataset = load_dataset("worldboss/ewe_bible_v2_tts")
dataset = load_dataset("Elbosso23/ewe_bible_30_sec_sample")
train_test_split = dataset["train"].train_test_split(test_size=0.2)

# Nouveau dataset avec train et test
common_voice  = DatasetDict({
    "train": train_test_split["train"].shuffle(seed=42).select(range(16)),
    "test": train_test_split["test"].shuffle(seed=42).select(range(4))
})



/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [4]:
common_voice

DatasetDict({
    train: Dataset({
        features: ['Id', 'audio', 'transcription'],
        num_rows: 16
    })
    test: Dataset({
        features: ['Id', 'audio', 'transcription'],
        num_rows: 4
    })
})

In [5]:
common_voice = common_voice.select_columns(["audio", "transcription"])

In [6]:
from transformers.models.whisper.tokenization_whisper import TO_LANGUAGE_CODE

TO_LANGUAGE_CODE

{'english': 'en',
 'chinese': 'zh',
 'german': 'de',
 'spanish': 'es',
 'russian': 'ru',
 'korean': 'ko',
 'french': 'fr',
 'japanese': 'ja',
 'portuguese': 'pt',
 'turkish': 'tr',
 'polish': 'pl',
 'catalan': 'ca',
 'dutch': 'nl',
 'arabic': 'ar',
 'swedish': 'sv',
 'italian': 'it',
 'indonesian': 'id',
 'hindi': 'hi',
 'finnish': 'fi',
 'vietnamese': 'vi',
 'hebrew': 'he',
 'ukrainian': 'uk',
 'greek': 'el',
 'malay': 'ms',
 'czech': 'cs',
 'romanian': 'ro',
 'danish': 'da',
 'hungarian': 'hu',
 'tamil': 'ta',
 'norwegian': 'no',
 'thai': 'th',
 'urdu': 'ur',
 'croatian': 'hr',
 'bulgarian': 'bg',
 'lithuanian': 'lt',
 'latin': 'la',
 'maori': 'mi',
 'malayalam': 'ml',
 'welsh': 'cy',
 'slovak': 'sk',
 'telugu': 'te',
 'persian': 'fa',
 'latvian': 'lv',
 'bengali': 'bn',
 'serbian': 'sr',
 'azerbaijani': 'az',
 'slovenian': 'sl',
 'kannada': 'kn',
 'estonian': 'et',
 'macedonian': 'mk',
 'breton': 'br',
 'basque': 'eu',
 'icelandic': 'is',
 'armenian': 'hy',
 'nepali': 'ne',
 'mongol

In [ ]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained(
    "openai/whisper-small", language="yo", task="transcribe"
)

In [9]:
common_voice["train"].features

{'audio': Audio(sampling_rate=None, decode=True, stream_index=None),
 'transcription': Value('string')}

In [10]:
from datasets import Audio

sampling_rate = processor.feature_extractor.sampling_rate
common_voice = common_voice.cast_column("audio", Audio(sampling_rate=sampling_rate))

In [11]:
def prepare_dataset(example):
    audio = example["audio"]

    example = processor(
        audio=audio["array"],
        sampling_rate=audio["sampling_rate"],
        text=example["transcription"],
    )

    # compute input length of audio sample in seconds
    example["input_length"] = len(audio["array"]) / audio["sampling_rate"]

    return example

In [13]:
# Installer version compatible si besoin
!pip install --upgrade torchcodec==0.8.1  # ou choisir version qui correspond à torch
!pip install --upgrade torch==2.9.0         # exemple, si tu veux torch 2.9
# puis redémarrer le runtime (Runtime ▶ Restart runtime) dans Colab

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 40.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 115.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 60.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 76.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5/267.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.3/322.3 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.

In [12]:
common_voice = common_voice.map(
    prepare_dataset, remove_columns=common_voice.column_names["train"], num_proc=1
)

Map:   0%|          | 0/16 [00:00<?, ? examples/s]

ImportError: To support decoding audio data, please install 'torchcodec'.

In [ ]:
max_input_length = 30.0


def is_audio_in_length_range(length):
    return length < max_input_length

In [ ]:
common_voice["train"] = common_voice["train"].filter(
    is_audio_in_length_range,
    input_columns=["input_length"],
)

Filter:   0%|          | 0/16 [00:00<?, ? examples/s]

In [ ]:
common_voice["train"]

Dataset({
    features: ['input_features', 'labels', 'input_length'],
    num_rows: 16
})

In [ ]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union


@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(
        self, features: List[Dict[str, Union[List[int], torch.Tensor]]]
    ) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [
            {"input_features": feature["input_features"][0]} for feature in features
        ]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(
            labels_batch.attention_mask.ne(1), -100
        )

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [ ]:
import evaluate

metric = evaluate.load("wer")

In [ ]:
from transformers.models.whisper.english_normalizer import BasicTextNormalizer

normalizer = BasicTextNormalizer()


def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = processor.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = processor.batch_decode(label_ids, skip_special_tokens=True)

    # compute orthographic wer
    wer_ortho = 100 * metric.compute(predictions=pred_str, references=label_str)

    # compute normalised WER
    pred_str_norm = [normalizer(pred) for pred in pred_str]
    label_str_norm = [normalizer(label) for label in label_str]
    # filtering step to only evaluate the samples that correspond to non-zero references:
    pred_str_norm = [
        pred_str_norm[i] for i in range(len(pred_str_norm)) if len(label_str_norm[i]) > 0
    ]
    label_str_norm = [
        label_str_norm[i]
        for i in range(len(label_str_norm))
        if len(label_str_norm[i]) > 0
    ]

    wer = 100 * metric.compute(predictions=pred_str_norm, references=label_str_norm)

    return {"wer_ortho": wer_ortho, "wer": wer}

In [ ]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

In [ ]:
from functools import partial

# disable cache during training since it's incompatible with gradient checkpointing
model.config.use_cache = False

# set language and task for generation and re-enable cache
model.generate = partial(
    model.generate, language="yo", task="transcribe", use_cache=True
)

In [ ]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-small-dv",  # name on the HF Hub
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    lr_scheduler_type="constant_with_warmup",
    warmup_steps=50,
    max_steps=500,  # increase to 4000 if you have your own GPU or a Colab paid plan
    gradient_checkpointing=True,
    fp16=True,
    fp16_full_eval=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=16,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=500,
    eval_steps=500,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=False,
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=common_voice["train"],
    eval_dataset=common_voice["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor,
)

<ipython-input-24-6add8000ae4f>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
trainer.train()

Step,Training Loss,Validation Loss,Wer Ortho,Wer
500,0.000100,1.075951,52.173913,44.711538


You have passed task=transcribe, but also have set `forced_decoder_ids` to [[1, None], [2, 50359]] which creates a conflict. `forced_decoder_ids` will be ignored in favor of task=transcribe.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:2758: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 22520, 26130, 

TrainOutput(global_step=500, training_loss=0.16518038914212957, metrics={'train_runtime': 2182.8664, 'train_samples_per_second': 3.665, 'train_steps_per_second': 0.229, 'total_flos': 2.30868320256e+18, 'train_loss': 0.16518038914212957, 'epoch': 500.0})

In [ ]:
kwargs = {
    "dataset_tags": "Elbosso23/ewe_bible_30_sec_sample",
    "dataset": "Elbosso23/ewe_bible_30_sec_sample",  # a 'pretty' name for the training dataset
    "language": "dv",
    "model_name": "Whisper Small Dv - Ewe_ASR",  # a 'pretty' name for your model
    "finetuned_from": "openai/whisper-small",
    "tasks": "automatic-speech-recognition",
}

In [ ]:
# from transformers import pipeline

# pipe = pipeline("automatic-speech-recognition", model="Elbosso23/whisper-small-dv")

In [ ]:
common_voice

DatasetDict({
    train: Dataset({
        features: ['input_features', 'labels', 'input_length'],
        num_rows: 16
    })
    test: Dataset({
        features: ['input_features', 'labels', 'input_length'],
        num_rows: 4
    })
})

In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Détecter GPU ou CPU

# Envoyer le modèle sur le bon device
model = model.to(device)

# Sélectionner un exemple du dataset de test
sample = common_voice["test"][0]

# Charger les features correctement et envoyer sur le bon device
input_features = torch.tensor(sample["input_features"]).squeeze(0).unsqueeze(0).to(device)  # Correction ici
reference_ids = torch.tensor(sample["labels"]).to(device)  # Correction ici

# Passer dans le modèle pour prédire
with torch.no_grad():
    predicted_ids = model.generate(input_features)

# Décoder la sortie en texte
predicted_text = processor.batch_decode(predicted_ids.cpu(), skip_special_tokens=True)[0]  # Remettre sur CPU
reference_text = processor.batch_decode([reference_ids.cpu()], skip_special_tokens=True)[0]  # Remettre sur CPU

# Afficher la référence et la prédiction
print(f"🔹 Référence    : {reference_text}")
print(f"🔹 Prédiction   : {predicted_text}")


🔹 Référence    : Gbã la, wogblɔ nya siawo na Yudatɔwo ɖeɖe ko, emegbe la, wogblɔe na ame siwo menye Yudatɔwo o la hã. Nyanyui sia ɖe Mawu ƒe dzɔdzɔenyenye fia, dzɔdzɔenyenye si vana to xɔse me tso gɔmedzedze va se ɖe nuwuwu abe ale si Ŋɔŋlɔ Kɔkɔe la gblɔe ene be, Ame dzɔdzɔe anɔ agbe to xɔse me.
🔹 Prédiction   : Bã la, wogblɔ nyaseawo na yudatɔwo ɖeɖeko, emegbe la, wogblɔe na ame siwo menye yudatɔwo o la hã. Nyanyui sia ɖe Mawu ƒe dzorzönenye fia, dzorzönenye si vana to xɔse me tso gɔmedzedze vase ɖe nuvu, abe ale si ngɔglɔkɔkɔe la gblɔ ene be, ame dzorzɔe anɔ abe to xɔse me.


In [ ]:
# Sélectionner un exemple du dataset de test
sample = common_voice["test"][1]

# Charger les features correctement et envoyer sur le bon device
input_features = torch.tensor(sample["input_features"]).squeeze(0).unsqueeze(0).to(device)  # Correction ici
reference_ids = torch.tensor(sample["labels"]).to(device)  # Correction ici

# Passer dans le modèle pour prédire
with torch.no_grad():
    predicted_ids = model.generate(input_features)

# Décoder la sortie en texte
predicted_text = processor.batch_decode(predicted_ids.cpu(), skip_special_tokens=True)[0]  # Remettre sur CPU
reference_text = processor.batch_decode([reference_ids.cpu()], skip_special_tokens=True)[0]  # Remettre sur CPU

# Afficher la référence et la prédiction
print(f"🔹 Référence    : {reference_text}")
print(f"🔹 Prédiction   : {predicted_text}")


🔹 Référence    : Yesu Kristo, ƒe tsitretsitsi me la, Mawu fia be eyae nye Mawu ƒe Vi ŋusẽtɔ la, eye Mawu ŋutɔ ƒe nɔnɔme kɔkɔe la le eme.
🔹 Prédiction   :  Yesu Kristo ƒe titretitime la, Mawu ƒiã be eye nye Mawu ƒe vi ŋusẽ tola, eye Mawu ŋutɔƒe nɔnɔme kɔkɔe la le eme.


In [ ]:
# Sélectionner un exemple du dataset de test
sample = common_voice["test"][2]

# Charger les features correctement et envoyer sur le bon device
input_features = torch.tensor(sample["input_features"]).squeeze(0).unsqueeze(0).to(device)  # Correction ici
reference_ids = torch.tensor(sample["labels"]).to(device)  # Correction ici

# Passer dans le modèle pour prédire
with torch.no_grad():
    predicted_ids = model.generate(input_features)

# Décoder la sortie en texte
predicted_text = processor.batch_decode(predicted_ids.cpu(), skip_special_tokens=True)[0]  # Remettre sur CPU
reference_text = processor.batch_decode([reference_ids.cpu()], skip_special_tokens=True)[0]  # Remettre sur CPU

# Afficher la référence et la prédiction
print(f"🔹 Référence    : {reference_text}")
print(f"🔹 Prédiction   : {predicted_text}")


🔹 Référence    : Le esia ta Mawu tsɔ wo de asi na dzodzro vɔ̃wo; woƒe nyɔnuwo gɔ̃ hã tsɔ nu siwo menye dzɔdzɔmenuwo o la ɖɔli dzɔdzɔmenuwoe le gbɔdɔdɔnyawo gome. Nenema ke woƒe ŋutsuwo hã ɖe asi le dzɔdzɔme ƒe kadodo si le ŋutsu kple nyɔnu dome la ŋu, eye woƒe lãme fiena ɖe wo nɔewo ŋu. Ale ŋutsu wɔa ŋunyɔnu kple ŋutsu bubuwo. Ale woxɔa woƒe ŋukpenanuwo ƒe fetu ɖe woƒe ŋutilã me.
🔹 Prédiction   : Lé esia ta, Mawu tsɔwo de asi na dzodzɔvɔ̃ wòwé nyɔnuwò gɔ hã tzɔ nu si womenye dzodzɔmenuwo wòla ɖɔli dzodzɔmenuwoe. Le bɔdɔdɔnyawo gɔme, nenema ke wòfɔ ŋutsuwo hã reasi le dzodzɔmefɔe kadozɔ si le ŋutsu kple nyɔnu domela ŋu, eye wòfɔe lame fie nare wònɔe wòn, ale wòutsu woa ŋunyɔnu kple ŋutsu bubuwo, ale wòxɔa wòfɔe ŋutpe na nu wòfɔe fetu le wòfɔe ŋuti lame.


In [ ]:
# Sélectionner un exemple du dataset de test
sample = common_voice["test"][3]

# Charger les features correctement et envoyer sur le bon device
input_features = torch.tensor(sample["input_features"]).squeeze(0).unsqueeze(0).to(device)  # Correction ici
reference_ids = torch.tensor(sample["labels"]).to(device)  # Correction ici

# Passer dans le modèle pour prédire
with torch.no_grad():
    predicted_ids = model.generate(input_features)

# Décoder la sortie en texte
predicted_text = processor.batch_decode(predicted_ids.cpu(), skip_special_tokens=True)[0]  # Remettre sur CPU
reference_text = processor.batch_decode([reference_ids.cpu()], skip_special_tokens=True)[0]  # Remettre sur CPU

# Afficher la référence et la prédiction
print(f"🔹 Référence    : {reference_text}")
print(f"🔹 Prédiction   : {predicted_text}")


🔹 Référence    : Nye Paulo, Yesu Kristo ƒe dɔla, ame si wotia be wòanye mawunyadɔgbedela, eye wodɔe be wòaɖe gbeƒã Mawu ƒe nyanyui la. Mawu do Nyanyui sia ŋugbe xoxoxo to eƒe nyagblɔɖilawo dzi le Ŋɔŋlɔ Kɔkɔeawo me, tso Via, Yesu Kristo, ame si va xexea me abe amegbetɔ ene, eye wodzii abe David ƒe tɔgbuiyɔvi ene la ŋu.
🔹 Prédiction   : Enye fawlɔ, Yesu Kristo ƒe dɔ la, ame si wòtseya be wòanye Mawu dɔgbede la, eye wòdɔe be wòadegbe ƒe Mawu ƒe nyanyui la. Mawu ɖo nyanyui sia ŋu gbe xɔxɔ tɔe ƒe nyagbɔri lawo dzi le gɔnglɔkɔ kɔewo me. Tso via Yesu Kristo, ame si wòxexea me abe amegbetɔ ene, eye wò dzi abe davi ƒe tɔgbiwɔvi ene la ŋu.
